# Linux_elasticsearch
## 1. hosts file 수정
- host name 수정 : hostnamectl set -hostname elastic-1(/2/3)
- 터미널 : su root -> PS: bigdata -> gedit /etc/hosts 
- 아래 내용 추가 <br>
192.168.56.101 elastic-1 <br>
192.168.56.102 elastic-2 <br>
192.168.56.103 elastic-3 <br>

## 2. Elasticsearch & Kibana 설치
su root <br>
sudo yum -y install wget <br>

wget https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-7.14.1-linux-x86_64.tar.gz <br>
wget https://artifacts.elastic.co/downloads/kibana/kibana-7.14.1-linux-x86_64.tar.gz <br>

tar xfz elasticsearch-7.14.1-linux-x86_64.tar.gz <br>
tar xfz kibana-7.14.1-linux-x86_64.tar.gz <br>

mkdir files <br>
mv *.tar.gz files/ <br>

mv elasticsearch-7.14.1 es-714 <br>
mv kibana-7.14.1-linux-x86_64 kb-714 <br>


## 3. size 수정 (4g -> 1g)
gedit ~/es-714/config/jvm.options <br>


## 4. limits 파일 수정 (Linux)
sudo gedit /etc/security/limits.conf <br>

elastic-3	soft	nofile	65535 <br>
elastic-3	hard	nofile	65535 <br>
elastic-3	hard	nproc	65535 <br>
elastic-3	soft	nproc	65535 <br>
elastic-3	soft	memlock	unlimited <br>
elastic-3	hard	memlock	unlimited <br>


## 5. Linux 파일 수정
sudo gedit /etc/security/limits.d/20-nproc.conf <br>
- 추가 (기존 내용 삭제 - 밑에서 2번째 줄)<br>
*	soft	nofile	65535 <br>
*	hard	nofile	65535 <br>
*	hard	nproc	65535 <br>
*	soft	nproc	65535 <br>
*	soft	memlock	unlimited <br>
*	hard	memlock	unlimited <br>

## Linux 파일 수정
sudo gedit /etc/sysctl.conf <br>
- 추가
vm.max_map_count=262144 <br>

## Linux reboot
sudo shutdown -r now

## 6. elasticsearch.yml 파일 수정 (각 노드별로 수행)
gedit ~/es-714/config/elasticsearch.yml <br>

#Node 1
cluster.name: "es-cluster"
node.name: "node-1"
#network.host: "192.168.56.101"
network.host: ["_local_", "_site_"]
discovery.seed_hosts: ["192.168.56.101", "192.168.56.102", "192.168.56.103"]
cluster.initial_master_nodes: ["node-1", "node-2", "node-3"]
xpack.security.enabled: true
xpack.security.transport.ssl.enabled: true
xpack.security.transport.ssl.keystore.path: certs/es-cluster.p12
xpack.security.transport.ssl.truststore.path: certs/es-cluster.p12


#Node 2
cluster.name: "es-cluster"
node.name: "node-2"
#network.host: "192.168.56.102"
network.host: ["_local_", "_site_"]
discovery.seed_hosts: ["192.168.56.101", "192.168.56.102", "192.168.56.103"]
cluster.initial_master_nodes: ["node-1", "node-2", "node-3"]
xpack.security.enabled: true
xpack.security.transport.ssl.enabled: true
xpack.security.transport.ssl.keystore.path: certs/es-cluster.p12
xpack.security.transport.ssl.truststore.path: certs/es-cluster.p12


#Node 3
cluster.name: "es-cluster"
node.name: "node-3"
#network.host: "192.168.56.103"
network.host: ["_local_", "_site_"]
discovery.seed_hosts: ["192.168.56.101", "192.168.56.102", "192.168.56.103"]
cluster.initial_master_nodes: ["node-1", "node-2", "node-3"]
xpack.security.enabled: true
xpack.security.transport.ssl.enabled: true
xpack.security.transport.ssl.keystore.path: certs/es-cluster.p12
xpack.security.transport.ssl.truststore.path: certs/es-cluster.p12

## 7. Elasticsearch & Kibana 실행 하기 위해 port 열기
sudo su
firewall-cmd --permanent --zone=public --add-port=9200/tcp
firewall-cmd --permanent --zone=public --add-port=9300/tcp
firewall-cmd --reload
firewall-cmd --list-all

## 3번째 노드에만 Kibana 실행 하기 위함
firewall-cmd --permanent --zone=public --add-port=5601/tcp
firewall-cmd --reload
firewall-cmd --list-all
exit

## elasticsearch 통신을 위한 파일 생성
~/es-714/bin/elasticsearch-certutil ca
<br>
mkdir ~/es-714/config/certs
<br>
~/es-714/bin/elasticsearch-certutil cert --ca elastic-stack-ca.p12 --ip 192.168.56.101,192.168.56.102,192.168.56.103 --out ~/es-714/config/certs/es-cluster.p12
<br>
- 파일 확인 <br>
ls ~/es-714/config/certs/


## 파일 배포하기
- 파일 디렉토리 만들기 (2, 3번 노드에서) <br>
mkdir ~/es-714/config/certs 
- 파일 배포 (1번 노드에서) <br>
scp -r ~/es-714/config/certs/* elastic-2@192.168.56.102:~/es-714/config/certs/  <br>
scp -r ~/es-714/config/certs/* elastic-3@192.168.56.103:~/es-714/config/certs/

## 보안 관련 설정 (모든 노드에서)
- 파일 생성 <br>
~/es-714/bin/elasticsearch-keystore create
<br>
~/es-714/bin/elasticsearch-keystore add xpack.security.transport.ssl.keystore.secure_password
<br>
~/es-714/bin/elasticsearch-keystore add xpack.security.transport.ssl.truststore.secure_password
<br>
- 파일 확인 <br>
~/es-714/bin/elasticsearch-keystore list
<br>

## elasticsearch setup & 실행
- elasticsearch 실행 (모든 노드에서) <br>
~/es-714/bin/elasticsearch

- (노드1에서만) <br>
~/es-714/bin/elasticsearch-setup-passwords interactive

- 실행 확인 (master 노드는 랜덤하게 선정) <br>
curl "elastic-1:9200/_cat/nodes?v" -u elastic:bigdata

- 어느 노드에서든 Query 가능 <br>
curl -XGET elastic-1:9200 -u elastic:bigdata <br>
curl -XGET elastic-2:9200 -u elastic:bigdata <br>
curl -XGET elastic-3:9200 -u elastic:bigdata <br>


## Kibana setup (3번 노드에서만)
gedit ~/kb-714/config/kibana.yml <br>

- yml 파일 내용 추가 <br>
server.host: "192.168.56.103"
server.name: "my-kibana"
elasticsearch.hosts: ["http://192.168.56.103:9200"]
elasticsearch.username: "kibana_system"

- 암호 설정  <br>
~/kb-714/bin/kibana-keystore create <br>
~/kb-714/bin/kibana-keystore add elasticsearch.password <br>
~/kb-714/bin/kibana-keystore list <br>

- firefox update <br>
sudo yum -y update firefox

- Kibana 실행 <br>
~/kb-714/bin/kibana

- firefox에서 접속: 192.168.56.103:5601 <br>
ID : elastic <br>
PS : bigdata <br>
